In [1]:
from xlrd import open_workbook
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn import metrics
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import svm
#import xgboost
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import roc_auc_score
import lasagne
import theano
import theano.tensor as T
from lasagne import layers as L
import collections
import os
import pandas as pd
import networkx as nx
import math
import re
from difflib import SequenceMatcher
from unidecode import unidecode 
from openpyxl import Workbook

In [2]:
Graph_train =nx.read_gml("/Users/olgagerasimova/Desktop/Data/Graph_train.gml")

In [3]:
Graph_2017=nx.read_gml("/Users/olgagerasimova/Desktop/Data/Graph_2017.gml")

In [4]:
def iterate_minibatches(X, y, batchsize):

    indices = np.arange(len(X))
    np.random.shuffle(indices)
    for start_idx in range(0, len(X) - batchsize + 1, batchsize):
        excerpt = indices[start_idx:start_idx + batchsize]
        yield X[excerpt], y[excerpt]
   

In [5]:
def models(X_train, y_train, X_test, y_test):
    MSE_train= []
    CV_train = []
    MSE_test= []
    CV_test = []
    MAE_train = []
    MAPE_train =[]
    MAE_test = []
    MAPE_test =[]
    
    # 1.
    # Create linear regression object
    regr = linear_model.LinearRegression()

    # Train the model using the training sets
    regr.fit(X_train, y_train)

    y_pred_train = regr.predict(X_train)
    y_pred_test =regr.predict(X_test)
    
    # The mean squared error
    MSE_train.append(np.mean(( y_pred_train - y_train) ** 2))
    MSE_test.append(np.mean((y_pred_test - y_test) ** 2))

    # The mean score and the 95% confidence interval of the score estimate are hence given by:
    scores1 = cross_val_score(regr, X_train, y_train, cv=10)
    CV_train.append([scores1.mean(), scores1.std() * 2])
    scores2 = cross_val_score(regr, X_test, y_test, cv=10)
    CV_test.append([scores2.mean(), scores2.std() * 2])
    
    MAE_train.append(metrics.mean_absolute_error(y_train, y_pred_train))
    MAE_test.append(metrics.mean_absolute_error(y_test, y_pred_test))
    
    MAPE_train.append(np.mean(np.abs((y_train - y_pred_train) / y_train)) * 100)
    MAPE_test.append(np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100)
    
    print 1
    
    # 2. Lasso
    lasso = Lasso(alpha = 0.016)
    y_pred_lasso_train = lasso.fit(X_train, y_train).predict(X_train)
    y_pred_lasso = lasso.fit(X_train, y_train).predict(X_test)
    
    MSE_train.append(np.mean((y_pred_lasso_train - y_train) ** 2))
    MSE_test.append(np.mean((y_pred_lasso - y_test) ** 2))
     
    scores1 = cross_val_score(lasso, X_train, y_train, cv=10)
    CV_train.append([scores1.mean(), scores1.std() * 2])
    scores2 = cross_val_score(lasso, X_test, y_test, cv=10)
    CV_test.append([scores2.mean(), scores2.std() * 2])
    
    MAE_train.append(metrics.mean_absolute_error(y_train, y_pred_lasso_train))
    MAE_test.append(metrics.mean_absolute_error(y_test, y_pred_lasso))
    
    MAPE_train.append(np.mean(np.abs((y_train - y_pred_lasso_train) / y_train)) * 100)
    MAPE_test.append(np.mean(np.abs((y_test - y_pred_lasso) / y_test)) * 100)

    print 2
    
    # 3. SVR
    svr = svm.SVR()
    svr.fit(X_train, y_train)
                     
    y_pred_train = svr.predict(X_train)
    y_pred_test = svr.predict(X_test)
    
    MSE_train.append(np.mean((y_pred_train - y_train) ** 2))
    MSE_test.append(np.mean((y_pred_test - y_test) ** 2))
    
    scores1 = cross_val_score(svr, X_train, y_train, cv=10)
    CV_train.append([scores1.mean(), scores1.std() * 2])
    scores2 = cross_val_score(svr, X_test, y_test, cv=10)
    CV_test.append([scores2.mean(), scores2.std() * 2])
    
    MAE_train.append(metrics.mean_absolute_error(y_train, y_pred_train))
    MAE_test.append(metrics.mean_absolute_error(y_test, y_pred_test))
    
    MAPE_train.append(np.mean(np.abs((y_train - y_pred_train) / y_train)) * 100)
    MAPE_test.append(np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100)
    
    print 3
    
    # 4. 
    params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
              'learning_rate': 0.01, 'loss': 'ls'}
    clf = GradientBoostingRegressor(**params)
    clf.fit(X_train, y_train)
                     
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)   
                     
    MSE_train.append(np.mean((y_pred_train - y_train) ** 2))
    MSE_test.append(np.mean((y_pred_test - y_test) ** 2))
    
    scores1 = cross_val_score( clf, X_train, y_train, cv=10)
    CV_train.append([scores1.mean(), scores1.std() * 2])
    scores2 = cross_val_score( clf, X_test, y_test, cv=10)
    CV_test.append([scores2.mean(), scores2.std() * 2])
    
    MAE_train.append(metrics.mean_absolute_error(y_train, y_pred_train))
    MAE_test.append(metrics.mean_absolute_error(y_test, y_pred_test))
    
    MAPE_train.append(np.mean(np.abs((y_train - y_pred_train) / y_train)) * 100)
    MAPE_test.append(np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100)
    print 4
    
    # 5.    
    return MSE_train, CV_train, MSE_test, CV_test, MAE_train, MAPE_train, MAE_test, MAPE_test 

In [6]:
def model(X_train, y_train, X_test, y_test):
    MSE_train= []
    CV_train = []
    MSE_test= []
    CV_test = []
    MAE_train = []
    MAPE_train =[]
    MAE_test = []
    MAPE_test =[]
    
    #  Lasso
    lasso = Lasso(alpha = 0.016)
    y_pred_lasso_train = lasso.fit(X_train, y_train).predict(X_train)
    y_pred_lasso = lasso.fit(X_train, y_train).predict(X_test)
    
    MSE_train.append(np.mean((y_pred_lasso_train - y_train) ** 2))
    MSE_test.append(np.mean((y_pred_lasso - y_test) ** 2))
     
    scores1 = cross_val_score(lasso, X_train, y_train, cv=10)
    CV_train.append([scores1.mean(), scores1.std() * 2])
    scores2 = cross_val_score(lasso, X_test, y_test, cv=10)
    CV_test.append([scores2.mean(), scores2.std() * 2])
    
    MAE_train.append(metrics.mean_absolute_error(y_train, y_pred_lasso_train))
    MAE_test.append(metrics.mean_absolute_error(y_test, y_pred_lasso))
    
    MAPE_train.append(np.mean(np.abs((y_train - y_pred_lasso_train) / y_train)) * 100)
    MAPE_test.append(np.mean(np.abs((y_test - y_pred_lasso) / y_test)) * 100)

    return MSE_train[0], CV_train[0], MSE_test[0], CV_test[0], MAE_train[0], MAPE_train[0], MAE_test[0], MAPE_test[0] 

In [29]:
def model_svr(X_train, y_train, X_test, y_test):
    MSE_train= []
    CV_train = []
    MSE_test= []
    CV_test = []
    MAE_train = []
    MAPE_train =[]
    MAE_test = []
    MAPE_test =[]
    
    # SVR
    svr = svm.SVR()
    svr.fit(X_train, y_train)
                     
    y_pred_train = svr.predict(X_train)
    y_pred_test = svr.predict(X_test)
    
    MSE_train.append(np.mean((y_pred_train - y_train) ** 2))
    MSE_test.append(np.mean((y_pred_test - y_test) ** 2))
    
    scores1 = cross_val_score(svr, X_train, y_train, cv=10)
    CV_train.append([scores1.mean(), scores1.std() * 2])
    scores2 = cross_val_score(svr, X_test, y_test, cv=10)
    CV_test.append([scores2.mean(), scores2.std() * 2])
    
    MAE_train.append(metrics.mean_absolute_error(y_train, y_pred_train))
    MAE_test.append(metrics.mean_absolute_error(y_test, y_pred_test))
    
    MAPE_train.append(np.mean(np.abs((y_train - y_pred_train) / y_train)) * 100)
    MAPE_test.append(np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100)


    return MSE_train[0], CV_train[0], MSE_test[0], CV_test[0], MAE_train[0], MAPE_train[0], MAE_test[0], MAPE_test[0], y_pred_train, y_pred_test   

In [130]:
MSE_train= []
CV_train = []
MSE_test= []
CV_test = []
MAE_train = []
MAPE_train =[]
MAE_test = []
MAPE_test =[]
values = [0.25,0.5,1,2,4]
for x in values:
    for y in values:
        X_train = []
        y_train = []
        X_test = []
        y_test = []
        f_emb_train = open('/Users/olgagerasimova/Desktop/Data/emb/out'+str(x)+'_'+str(y)+'.txt')
        f_emb_2017 = open('/Users/olgagerasimova/Desktop/Data/emb/out_x'+str(x)+'_'+str(y)+'.txt')
        emb_train = []
        emb_2017 = []
        for line in f_emb_train:
            emb_train.append(line.split(' '))
        for line in f_emb_2017:
            emb_2017.append(line.split(' '))
        f_emb_train.close()
        f_emb_2017.close()
        for i in range(len(emb_2017)):
            emb_2017[i][len(emb_2017[i])-1] = emb_2017[i][len(emb_2017[i])-1][:-1]
        for i in range(len(emb_train)):
            emb_train[i][len(emb_train[i])-1] = emb_train[i][len(emb_train[i])-1][:-1]
        emb_train_dict = {} 
        for i in range(1,len(emb_train)):
            emb_train_dict[int(emb_train[i][0])] = [float(j) for j in emb_train[i][1:]]
        
        for e in edges_2017:
            emb1 = emb_2017_dict[int(e[0])]
            emb2 = emb_2017_dict[int(e[1])]
            w = float(e[2])
            y_test.append(w)
            res = avg_sum(emb1, emb2)
            X_test.append(res)
        for e in edges_train:
            emb1 = emb_train_dict[int(e[0])]
            emb2 = emb_train_dict[int(e[1])]
            w = float(e[2])
            y_train.append(w)
            res = avg_sum(emb1, emb2)
            X_train.append(res)
        mse_train, cv_train, mse_test,cv_test, mae_train, mape_train, mae_test,mape_test = model(X_train, y_train, X_test, y_test)
        MSE_train.append(mse_train)
        CV_train.append( cv_train)
        MSE_test.append(mse_test)
        CV_test.append(cv_test)
        MAE_train.append(mae_train)
        MAPE_train.append(mape_train)
        MAE_test.append(mae_test)
        MAPE_test.append(mape_test)
        print x,y

0.25 0.25
0.25 0.5
0.25 1
0.25 2
0.25 4
0.5 0.25
0.5 0.5
0.5 1
0.5 2
0.5 4
1 0.25
1 0.5
1 1
1 2
1 4
2 0.25
2 0.5
2 1
2 2
2 4
4 0.25
4 0.5
4 1
4 2
4 4


In [131]:
f_res = open('/Users/olgagerasimova/Desktop/Data/emb/res_pq_lasso.txt','w')
for i in range(len(MSE_train)):
    f_res.write(str(MSE_train[i])+"  "+str(CV_train[i][0]) +"  "+ str(CV_train[i][1])+"  "+str(MSE_test[i])+"  "\
                +str(CV_train[i][0]) +"  "+ str(CV_train[i][1])+"  "+str(MAE_train[i])+"  "+str(MAPE_train[i]) +"  "\
                + str(MAE_test[i])+ "  "+str(MAPE_test[i])+'\n')
f_res.close()

In [132]:
np.argmin(MSE_train) # q=2 p = 0.25

16

In [133]:
np.argmin(MSE_test) # q=4 p = 0.5

22

In [8]:
f_train = open('/Users/olgagerasimova/Desktop/Data/edgelist_train.txt')
f_2017 = open('/Users/olgagerasimova/Desktop/Data/edgelist_2017.txt')
#f_emb_train = open('/Users/olgagerasimova/Downloads/node2vec-master/emb/out.txt')
#f_emb_2017 = open('/Users/olgagerasimova/Downloads/node2vec-master/emb/out_x.txt')

In [9]:
edges_train = []
edges_2017 = []
#emb_train = []
#emb_2017 = []

In [10]:
for line in f_train:
    edges_train.append(line.split(' '))

In [11]:
for line in f_2017:
    edges_2017.append(line.split(' '))

In [8]:
#for line in f_emb_train:
#    emb_train.append(line.split(' '))

In [9]:
#for line in f_emb_2017:
#    emb_2017.append(line.split(' '))

In [12]:
f_train.close()
f_2017.close()

#f_emb_train.close()
#f_emb_2017.close()

In [13]:
for i in range(len(edges_train)):
    edges_train[i][2] = edges_train[i][2][:-1]

In [14]:
for i in range(len(edges_2017)):
    edges_2017[i][2] = edges_2017[i][2][:-1]

In [13]:
for i in range(len(emb_2017)):
    emb_2017[i][len(emb_2017[i])-1] = emb_2017[i][len(emb_2017[i])-1][:-1]

In [14]:
emb_2017_dict = {} 
for i in range(1,len(emb_2017)):
    emb_2017_dict[int(emb_2017[i][0])] = [float(j) for j in emb_2017[i][1:]]

In [15]:
for i in range(len(emb_train)):
    emb_train[i][len(emb_train[i])-1] = emb_train[i][len(emb_train[i])-1][:-1]

In [17]:
emb_train_dict = {} 
for i in range(1,len(emb_train)):
    emb_train_dict[int(emb_train[i][0])] = [float(j) for j in emb_train[i][1:]]

In [15]:
X_train = []
y_train = []
X_test = []
y_test = []

In [28]:
for e in edges_2017:
    emb1 = emb_2017_dict[int(e[0])]
    emb2 = emb_2017_dict[int(e[1])]
    w = float(e[2])
    y_test.append(w)
    res = avg_sum(emb1, emb2)
    X_test.append(res)

In [34]:
for e in edges_train:
    emb1 = emb_train_dict[int(e[0])]
    emb2 = emb_train_dict[int(e[1])]
    w = float(e[2])
    y_train.append(w)
    res = avg_sum(emb1, emb2)
    X_train.append(res)

In [42]:
MSE_train, CV_train, MSE_test,CV_test, MAE_train, MAPE_train,MAE_test,MAPE_test = \
models(X_train, y_train, X_test, y_test)

1
2
3


In [55]:
MSE_train

[0.2990588347441652,
 0.3822489548219429,
 0.3475414396575275,
 0.1782310129713387,
 0.1782310129713387]

In [56]:
CV_train

[[0.2258244038344645, 0.08042779700926814],
 [0.04944167326898944, 0.01649577553479001],
 [0.12344459198805668, 0.050446954281378596],
 [0.3980763174473421, 0.07767067652565524]]

In [57]:
MAPE_train

[61.86817255165805, 54.836067146362865, 24.301688341316627, 34.83424604824781]

In [58]:
MAE_train

[0.28559030450915557,
 0.293343695971142,
 0.19933392581272866,
 0.186962940200237]

In [59]:
MSE_test

[5.778435590487275,
 1.2943507537317454,
 1.4672382678414462,
 1.3596406539884769,
 1.342760247602434]

In [60]:
CV_test

[[0.2703363189890003, 0.6153727033789264],
 [0.2446658738034529, 0.11572054870743606],
 [0.1983650843310501, 0.10974959870475084],
 [0.6116718588969366, 0.1323381494384997]]

In [61]:
MAE_test

[2.0688574786999347,
 0.5237419272914334,
 0.5754998686585696,
 0.8974951827990754]

In [62]:
MAPE_test

[464.16744908568734, 42.1997312045351, 49.810900548328846, 201.03229282768888]

In [99]:
# https://github.com/aditya-grover/node2vec
values = [0.25,0.5,1,2,4] # d = 128
for x in values:
    for y in values:
        %run /Users/olgagerasimova/Downloads/node2vec-master/src/main.py --input /Users/olgagerasimova/Downloads/node2vec-master/graph/edgelist_train.txt --output /Users/olgagerasimova/Desktop/Data/emb/out{x}_{y}.txt --q {x} --p {y} --weighted
        %run /Users/olgagerasimova/Downloads/node2vec-master/src/main.py --input /Users/olgagerasimova/Downloads/node2vec-master/graph/edgelist_2017.txt --output /Users/olgagerasimova/Desktop/Data/emb/out_x{x}_{y}.txt --q {x} --p {y} --weighted

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10

In [19]:
values_d = [16,32,64,128,256]
for d in values_d:
    %run /Users/olgagerasimova/Downloads/node2vec-master/src/main.py --input /Users/olgagerasimova/Downloads/node2vec-master/graph/edgelist_train.txt --output /Users/olgagerasimova/Desktop/Data/emb/out_d_{d}.txt --dimensions {d} --q 2 --p 0.25 --weighted
    %run /Users/olgagerasimova/Downloads/node2vec-master/src/main.py --input /Users/olgagerasimova/Downloads/node2vec-master/graph/edgelist_2017.txt --output /Users/olgagerasimova/Desktop/Data/emb/out_x_d_{d}.txt --dimensions {d} --q 2 --p 0.25 --weighted

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [26]:
# edge embbeding functions
def avg_sum(v1, v2):
    return (np.array(v1)+np.array(v2))/2
def mult(v1, v2):
    return np.array(v1)*np.array(v2)
def w_l1(v1, v2):
    return np.abs(np.array(v1)-np.array(v2))
def w_l2(v1, v2):
    return (np.array(v1)-np.array(v2))**2
def nw_l1(v1, v2, graph, n1, n2, embs ):
    neig1 = [n for n in graph.neighbors(str(n1))]
    neig2 = [n for n in graph.neighbors(str(n2))]
    sum1 = np.zeros(len(v1))
    sum2 = np.zeros(len(v2))
    for n in neig1:
        sum1 += np.array(embs[int(n)])
    for n in neig2:
        sum2 += np.array(embs[int(n)])
    return np.abs((sum1+np.array(v1))/(len(neig1)+1)+(sum2+np.array(v2))/(len(neig2)+1))
def nw_l2(v1, v2, graph, n1, n2, embs ):
    neig1 = [n for n in graph.neighbors(str(n1))]
    neig2 = [n for n in graph.neighbors(str(n2))]
    sum1 = np.zeros(len(v1))
    sum2 = np.zeros(len(v2))
    for n in neig1:
        sum1 += np.array(embs[int(n)])
    for n in neig2:
        sum2 += np.array(embs[int(n)])
    return ((sum1+np.array(v1))/(len(neig1)+1)+(sum2+np.array(v2))/(len(neig2)+1))**2

In [27]:
MSE_train= []
CV_train = []
MSE_test= []
CV_test = []
MAE_train = []
MAPE_train =[]
MAE_test = []
MAPE_test =[]
values_d = [16,32,64,128,256]
for d in values_d:
        X_train = []
        y_train = []
        X_test = []
        y_test = []
        f_emb_train = open('/Users/olgagerasimova/Desktop/Data/emb/out_d_'+str(d)+'.txt')
        f_emb_2017 = open('/Users/olgagerasimova/Desktop/Data/emb/out_x_d_'+str(d)+'.txt')
        emb_train = []
        emb_2017 = []
        for line in f_emb_train:
            emb_train.append(line.split(' '))
        for line in f_emb_2017:
            emb_2017.append(line.split(' '))
        f_emb_train.close()
        f_emb_2017.close()
        for i in range(len(emb_2017)):
            emb_2017[i][len(emb_2017[i])-1] = emb_2017[i][len(emb_2017[i])-1][:-1]
        emb_2017_dict = {} 
        for i in range(1,len(emb_2017)):
            emb_2017_dict[int(emb_2017[i][0])] = [float(j) for j in emb_2017[i][1:]]
        for i in range(len(emb_train)):
            emb_train[i][len(emb_train[i])-1] = emb_train[i][len(emb_train[i])-1][:-1]
        emb_train_dict = {} 
        for i in range(1,len(emb_train)):
            emb_train_dict[int(emb_train[i][0])] = [float(j) for j in emb_train[i][1:]]
        
        for e in edges_2017:
            emb1 = emb_2017_dict[int(e[0])]
            emb2 = emb_2017_dict[int(e[1])]
            w = float(e[2])
            y_test.append(w)
            res = avg_sum(emb1, emb2) # !!!  use different functions
            X_test.append(res)
        for e in edges_train:
            emb1 = emb_train_dict[int(e[0])]
            emb2 = emb_train_dict[int(e[1])]
            w = float(e[2])
            y_train.append(w)
            res = avg_sum(emb1, emb2) # !!!
            X_train.append(res)
        mse_train, cv_train, mse_test,cv_test, mae_train, mape_train, mae_test,mape_test = model_svr(X_train, y_train, X_test, y_test)
        MSE_train.append(mse_train)
        CV_train.append( cv_train)
        MSE_test.append(mse_test)
        CV_test.append(cv_test)
        MAE_train.append(mae_train)
        MAPE_train.append(mape_train)
        MAE_test.append(mae_test)
        MAPE_test.append(mape_test)
        print d
        X_train = []
        y_train = []
        X_test = []
        y_test = []

        
        for e in edges_2017:
            emb1 = emb_2017_dict[int(e[0])]
            emb2 = emb_2017_dict[int(e[1])]
            w = float(e[2])
            y_test.append(w)
            res = mult(emb1, emb2) # !!! use different functions
            X_test.append(res)
        for e in edges_train:
            emb1 = emb_train_dict[int(e[0])]
            emb2 = emb_train_dict[int(e[1])]
            w = float(e[2])
            y_train.append(w)
            res = mult(emb1, emb2) # !!!
            X_train.append(res)
        mse_train, cv_train, mse_test,cv_test, mae_train, mape_train, mae_test,mape_test = model_svr(X_train, y_train, X_test, y_test)
        MSE_train.append(mse_train)
        CV_train.append( cv_train)
        MSE_test.append(mse_test)
        CV_test.append(cv_test)
        MAE_train.append(mae_train)
        MAPE_train.append(mape_train)
        MAE_test.append(mae_test)
        MAPE_test.append(mape_test)
        print d
        X_train = []
        y_train = []
        X_test = []
        y_test = []

        
        for e in edges_2017:
            emb1 = emb_2017_dict[int(e[0])]
            emb2 = emb_2017_dict[int(e[1])]
            w = float(e[2])
            y_test.append(w)
            res = w_l1(emb1, emb2) # !!! use different functions
            X_test.append(res)
        for e in edges_train:
            emb1 = emb_train_dict[int(e[0])]
            emb2 = emb_train_dict[int(e[1])]
            w = float(e[2])
            y_train.append(w)
            res = w_l1(emb1, emb2) # !!!
            X_train.append(res)
        mse_train, cv_train, mse_test,cv_test, mae_train, mape_train, mae_test,mape_test = model_svr(X_train, y_train, X_test, y_test)
        MSE_train.append(mse_train)
        CV_train.append( cv_train)
        MSE_test.append(mse_test)
        CV_test.append(cv_test)
        MAE_train.append(mae_train)
        MAPE_train.append(mape_train)
        MAE_test.append(mae_test)
        MAPE_test.append(mape_test)
        print d
        X_train = []
        y_train = []
        X_test = []
        y_test = []

        
        for e in edges_2017:
            emb1 = emb_2017_dict[int(e[0])]
            emb2 = emb_2017_dict[int(e[1])]
            w = float(e[2])
            y_test.append(w)
            res = w_l2(emb1, emb2) # !!! use different functions
            X_test.append(res)
        for e in edges_train:
            emb1 = emb_train_dict[int(e[0])]
            emb2 = emb_train_dict[int(e[1])]
            w = float(e[2])
            y_train.append(w)
            res = w_l2(emb1, emb2) # !!!
            X_train.append(res)
        mse_train, cv_train, mse_test,cv_test, mae_train, mape_train, mae_test,mape_test = model_svr(X_train, y_train, X_test, y_test)
        MSE_train.append(mse_train)
        CV_train.append( cv_train)
        MSE_test.append(mse_test)
        CV_test.append(cv_test)
        MAE_train.append(mae_train)
        MAPE_train.append(mape_train)
        MAE_test.append(mae_test)
        MAPE_test.append(mape_test)
        print d
        X_train = []
        y_train = []
        X_test = []
        y_test = []

        
        for e in edges_2017:
            emb1 = emb_2017_dict[int(e[0])]
            emb2 = emb_2017_dict[int(e[1])]
            w = float(e[2])
            y_test.append(w)
            res = nw_l1(emb1, emb2, Graph_2017,int(e[0]),int(e[1]),emb_2017_dict) # !!! use different functions
            X_test.append(res)
        for e in edges_train:
            emb1 = emb_train_dict[int(e[0])]
            emb2 = emb_train_dict[int(e[1])]
            w = float(e[2])
            y_train.append(w)
            res = nw_l1(emb1, emb2, Graph_train,int(e[0]),int(e[1]),emb_train_dict) # !!!
            X_train.append(res)
        mse_train, cv_train, mse_test,cv_test, mae_train, mape_train, mae_test,mape_test = model_svr(X_train, y_train, X_test, y_test)
        MSE_train.append(mse_train)
        CV_train.append( cv_train)
        MSE_test.append(mse_test)
        CV_test.append(cv_test)
        MAE_train.append(mae_train)
        MAPE_train.append(mape_train)
        MAE_test.append(mae_test)
        MAPE_test.append(mape_test)
        print d
        X_train = []
        y_train = []
        X_test = []
        y_test = []

        
        for e in edges_2017:
            emb1 = emb_2017_dict[int(e[0])]
            emb2 = emb_2017_dict[int(e[1])]
            w = float(e[2])
            y_test.append(w)
            res = nw_l2(emb1, emb2, Graph_2017,int(e[0]),int(e[1]),emb_2017_dict) # !!! use different functions
            X_test.append(res)
        for e in edges_train:
            emb1 = emb_train_dict[int(e[0])]
            emb2 = emb_train_dict[int(e[1])]
            w = float(e[2])
            y_train.append(w)
            res = nw_l2(emb1, emb2, Graph_train,int(e[0]),int(e[1]),emb_train_dict) # !!!
            X_train.append(res)
        mse_train, cv_train, mse_test,cv_test, mae_train, mape_train, mae_test,mape_test = model_svr(X_train, y_train, X_test, y_test)
        MSE_train.append(mse_train)
        CV_train.append( cv_train)
        MSE_test.append(mse_test)
        CV_test.append(cv_test)
        MAE_train.append(mae_train)
        MAPE_train.append(mape_train)
        MAE_test.append(mae_test)
        MAPE_test.append(mape_test)
        print d   

16
16
16
16
16
16
32
32
32
32
32
32
64
64
64
64
64
64
128
128
128
128
128
128
256
256
256
256
256
256


In [28]:
f_res_d = open('/Users/olgagerasimova/Desktop/Data/emb/res_d_svr.txt','w')
for i in range(len(MSE_train)):
    f_res_d.write(str(MSE_train[i])+"  "+str(CV_train[i][0]) +"  "+ str(CV_train[i][1])+"  "+str(MSE_test[i])+"  "\
                +str(CV_test[i][0]) +"  "+ str(CV_test[i][1])+"  "+str(MAE_train[i])+"  "+str(MAPE_train[i]) +"  "\
                + str(MAE_test[i])+ "  "+str(MAPE_test[i])+'\n')
f_res_d.close()

In [30]:
        X_train = []
        y_train = []
        X_test = []
        y_test = []
        f_emb_train = open('/Users/olgagerasimova/Desktop/Data/emb/out_d_'+str(16)+'.txt')
        f_emb_2017 = open('/Users/olgagerasimova/Desktop/Data/emb/out_x_d_'+str(16)+'.txt')
        emb_train = []
        emb_2017 = []
        for line in f_emb_train:
            emb_train.append(line.split(' '))
        for line in f_emb_2017:
            emb_2017.append(line.split(' '))
        f_emb_train.close()
        f_emb_2017.close()
        for i in range(len(emb_2017)):
            emb_2017[i][len(emb_2017[i])-1] = emb_2017[i][len(emb_2017[i])-1][:-1]
        emb_2017_dict = {} 
        for i in range(1,len(emb_2017)):
            emb_2017_dict[int(emb_2017[i][0])] = [float(j) for j in emb_2017[i][1:]]
        for i in range(len(emb_train)):
            emb_train[i][len(emb_train[i])-1] = emb_train[i][len(emb_train[i])-1][:-1]
        emb_train_dict = {} 
        for i in range(1,len(emb_train)):
            emb_train_dict[int(emb_train[i][0])] = [float(j) for j in emb_train[i][1:]]

        
        for e in edges_2017:
            emb1 = emb_2017_dict[int(e[0])]
            emb2 = emb_2017_dict[int(e[1])]
            w = float(e[2])
            y_test.append(w)
            res = nw_l2(emb1, emb2, Graph_2017,int(e[0]),int(e[1]),emb_2017_dict)
            X_test.append(res)
        for e in edges_train:
            emb1 = emb_train_dict[int(e[0])]
            emb2 = emb_train_dict[int(e[1])]
            w = float(e[2])
            y_train.append(w)
            res = nw_l2(emb1, emb2, Graph_train,int(e[0]),int(e[1]),emb_train_dict)
            X_train.append(res)
        mse_train, cv_train, mse_test,cv_test, mae_train, mape_train, mae_test,mape_test, y_pred_train, y_pred_test  = model_svr(X_train, y_train, X_test, y_test)



In [34]:
for i in y_test: 
    print i

2.4
3.0
0.3
2.26
0.3
3.0
0.3
0.3
3.0
1.65
0.3
2.0
1.65
0.3
4.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
4.0
0.3
3.0
0.3
0.3
0.3
0.3
0.3
2.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
3.0
0.3
0.3
0.3
2.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
2.15
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
1.15
3.0
0.3
0.3
0.3
0.3
0.3
0.3
4.0
4.0
0.3
0.3
0.3
1.07142857143
0.3
0.3
0.3
2.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
4.0
3.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
4.0
0.3
0.3
0.3
4.0
0.3
0.3
0.3
0.3
0.3
0.3
2.0
0.3
2.0
0.3
0.3
0.3
0.3
0.3
2.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
4.0
0.3
0.3
0.3
3.0
0.3
1.65
0.3
0.3
0.3
2.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
1.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
3.0
0.3
0.3
0.3
3.0
1.0
3.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
4.0
0.3
3.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
2.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
1.9
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3


0.3
0.3
0.3
0.3
0.3
0.3
4.0
0.3
0.3
0.3
0.3
0.3
0.3
4.0
0.3
0.3
0.3
0.3
2.0
0.3
0.3
0.3
0.3
0.3
3.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
4.0
4.0
0.3
0.3
2.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
3.0
0.3
0.3
3.0
0.3
0.3
0.3
0.3
0.3
3.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
4.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
4.0
0.3
0.3
0.3
1.0
0.3
0.3
0.3
4.0
0.3
0.3
3.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
1.0
0.3
0.3
0.3
4.0
0.3
0.3
0.3
0.3
0.3
2.0
0.3
2.15
0.3
0.3
0.3
0.3
0.3
0.3
4.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
4.0
4.0
4.0
0.3
0.3
0.3
0.3
1.65
0.3
0.3
0.3
0.3
2.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
3.0
4.0
0.3
0.3
0.3
0.3
0.3
3.0
0.3
2.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
4.0
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
0.3
3.0
0.3
0.3
4.0
0.3
0.3
0.3
4.0
0.3
0.3
0.3
0.3
0.3
0.3
2.0
0.3
1.65
0.3
0.3
0.3
0.3
0.3
0.3
0.3
3.0
2

In [33]:
for i in y_pred_train: 
    print i

0.34186193118962416
0.38919990664358695
0.4926066758577783
0.39963142164798127
0.37592998137924905
0.31622649972437666
0.39823882932372934
0.39683179727194395
0.3998946350035921
0.40042865999361194
0.3891483643322594
0.3997554395007705
0.3690241144558175
0.4002177022314961
0.3999897090147564
0.37272580595284477
0.3995534688542967
0.4000912423757144
0.21632764411362299
0.38213447688643615
0.37649934676788466
0.36965277091710047
0.3865563561268317
0.40007790214647054
0.39991255837807205
0.3906895259580464
0.23238046929839318
0.40011235672246853
0.3511635005209517
0.38410402846050873
0.40030892285549213
0.371983823175834
0.3738133790185516
0.3864094185938348
0.19977308050129827
0.3995154077970726
0.8467589930241679
0.2691376619909548
0.22621519876391732
0.39992110164581207
0.40015758875835283
1.1124668630074868
0.3999739058095344
0.38452170284792253
0.39075543919269057
0.21139558284467252
0.3484749715245522
0.3552178232581524
0.3888118656243697
0.35663609384790174
0.40008536761809366
0.40

0.39963194179098327
0.40027443206897695
0.38866762223734286
0.3815805207738968
0.39977671265344866
0.39197714584933624
0.3833470763303054
0.36707948345541047
0.5660719142619268
0.3613708577189
0.40000830084025346
0.37047150178836374
0.3945543736835346
1.789406984152345
0.3788929411823718
0.37531850438527314
0.37152382537508044
0.40016995138021344
0.40030562466856545
0.3770394897082613
0.33205017677058823
0.3996930377847192
0.22802541103700835
0.3666845287007468
0.2767998637004501
0.3258127179811879
0.40015596616713756
0.40024695722952164
0.40021316820891356
0.33241494137359673
0.4439425887054771
0.45284424460973216
0.4638381001418811
0.399894683165809
0.37364606766732944
0.4002688428423871
0.39980547867156035
0.3203802114845113
0.2715839289635597
0.399891613625581
0.3862206247582751
0.38859063633811847
0.24995681959755067
0.39992370704906544
0.4004602182529406
0.3998746295279507
0.40014280530316015
0.3977796972818729
0.35509740551429547
0.38674656207882063
0.36329781509599635
0.3201423

0.3062147693906092
0.3999374859752819
0.384231739024111
0.4000011163081153
0.39556323101114105
0.3515680725171304
0.2687670258718134
0.3821725443481938
0.4000045069575544
0.3618162025593179
0.3563294995526093
0.34572419918098674
0.3999376273046893
0.3878129182457205
0.20817613857459794
1.0504248507637304
0.39962490757451813
0.3983710807050604
0.4001375746741304
0.4001536927877307
0.37282968428020147
0.37821445903471873
0.4002625702998106
0.38545614195128025
0.4000539702772632
0.2606008756597174
0.40003252483069074
0.37649934676788466
0.37901598618761817
0.29219716501718773
0.4001686452919197
0.3717650124155941
0.4003424120879115
2.1199812130333013
0.388539804479054
0.3997599815086171
0.35929451235599175
0.3816333943873105
0.4004081314546333
0.3358961916826161
0.3585838922717299
0.3844376089105781
0.4002369304266604
0.3347770672804931
0.2699282411188035
0.40045414377802147
0.31114508561106713
0.37648953251410927
0.26341251855100845
0.3616281971590113
2.3815043101882223
0.392308291721307

0.24191783169022096
0.3999918674218651
0.39256125734407155
0.39964273379911996
0.3727269659473047
0.39981895385381344
0.5249955976072095
0.3842317390241109
0.3613035889156512
0.3682841134819379
0.3576381816783525
0.40002615671489217
0.3999495750932323
0.3932078116463507
0.40002500422859566
0.39982346671386826
0.45630514504010544
1.5891641603501552
0.39992744942246927
0.37950412688094587
0.39864186945094016
0.3930471918604196
0.3882618435407468
0.4000010890898055
0.40027465591463524
0.42478244447466756
0.4000384989956353
0.35831356057830255
0.3979085151510604
0.3940260254213044
0.31958038502094815
0.7924101932558443
0.3959727460305992
0.40036220282066737
0.37287603515608847
0.3999999774503119
0.40014282659351624
0.3941175602653594
0.39369443932113735
0.34572527341708237
0.37363476070841944
0.3948279344889212
0.36324313467793995
0.30018713554549614
0.3771807719386161
0.3783358383879353
0.2641103708707243
0.3759089147035284
0.4003672057949015
0.40002651669977274
0.38104126834213914
0.2833

0.35106758617518385
0.39097012097699935
0.38914357860427007
0.3437874968478585
0.39295479101028874
0.39962490757451813
0.2282426821888487
0.39998918817131235
0.23503541527090593
0.39385018773020464
0.3998200626682814
0.3950803294672975
0.39821021884244495
0.3997708801208313
0.3842017840502027
0.37649934676788466
0.3658474985199737
0.3507184924294876
0.40035168651997866
0.3996063570222148
0.3997706628474711
0.3707887089463737
0.20659933098943428
0.39907677027559246
0.39422162195835425
0.3761697550588211
0.3030547148334954
0.9233198102080689
0.27839453607427217
0.3647858004028033
0.5617596421751507
0.4001917733601537
0.39170582179340796
1.5067557318393106
0.3998598508906337
0.3646137297173141
0.3864038419631474
0.35836323465379727
0.4012610789927237
1.1700657896499713
0.3320608482892188
1.8998649122970492
0.3795686135509188
0.3419502448883124
0.3878782704797157
0.3922633739225404
0.3916169189389268
0.40032946459115215
0.39047593516381895
0.28446783488384686
0.40013440749627743
0.38481154

0.23059919919487742
0.3788076390444043
0.3786569236976437
0.26000923807734483
0.35011338593608
0.3996829178346526
0.39988226843840197
0.39482729764173635
0.3999107690956181
0.3942484427824632
0.3998273397758752
0.39284003781681315
0.38373743437705754
0.3806296406265778
0.3712006960858826
0.3999898865189783
0.400126413200392
0.3264341064423329
0.4003471333497895
0.3996117801017451
0.4000257252164815
0.4002924549746367
0.3929859813928396
0.39818783192310886
0.35160394779470877
0.4004579055018411
0.3999854653621682
0.3574086099623334
0.3040006358796732
0.7863070134598222
3.57618691253612
0.3999932788950842
0.4002404764662586
0.3996291257093719
0.37656967950720055
1.0999646658472544
0.22667435866752994
0.3996431596153374
0.40039698749235897
0.3754557446172285
0.3888982324661083
0.3893189611260741
0.3911697763753341
0.39863440278346257
0.3840383948878657
0.40003601039259823
0.37468337094109316
0.2940607426427144
0.3850880403291378
0.394211574300979
0.39972668828360725
0.37189498339885907
0.

0.3923867621455839
0.37296810221180265
0.4004473970028298
1.7981965600096141
1.8428614295216217
0.39642341369427353
0.37572745491968984
0.3999914291879943
0.21947064440842973
0.39498988499219334
0.40035636970751665
0.38841762596203044
0.3969213590416053
0.3957808451000091
0.4003937849125765
0.3804949617775251
0.5910726873052492
0.39996363041557004
0.2764677759567788
0.9366394069220807
0.39996845231352424
0.3998506595201445
0.28524420846989473
0.4004030141486586
0.4003190403787444
0.3999390698653096
0.39999215112939324
0.3739020962888718
1.5597586192575554
0.37504419917705545
0.40002517556855
0.4002106951513078
0.941796334155185
0.24961818392983487
1.5892246852618972
0.39669110422781845
0.39970542784248775
0.38693529664325155
0.3961676701540713
0.38170893469137
0.4004093613532158
0.38967746570783235
0.39999097214112245
0.36917550081422956
0.40407094791123455
0.2975623452531262
0.40026560958479923
0.22573932083488452
0.4001504299562726
0.36854347863678016
0.40005341320068527
0.3549543765

0.40027622108486405
0.2641833609424373
0.5692582870915357
0.2586236025040665
0.36787622219280613
0.42400228572531606
0.4001502010986329
0.3660486787178292
1.6120686168219847
0.2935645706279529
0.9378435465307273
0.48427319473056785
0.3315919026062876
0.40019340594748504
0.396322235839657
0.27440486409633397
0.3947888111073996
0.4002640189859432
0.4003105881087643
0.3803454005549483
0.392386762145584
0.3972582753617781
0.40029932781996647
0.2009824847799403
0.3996249075745178
0.19961122845993112
0.39477748658953116
0.3895013326152714
0.4000717365192876
0.36708005620007617
0.375832379019489
0.39996004188061784
0.39974051389296883
0.39994252695343907
0.3877046851232654
0.40006752716341865
0.3856880095144857
0.38116257706330464
0.33240244933996144
0.3882920264843005
0.3973760173952785
0.9564805917260815
0.39125973879210985
0.40029271269673244
0.3996336051906947
0.33206084828921956
0.3783358383879353
0.31453500948044416
0.3938253489683
0.3812919963355178
0.32145090457573966
0.37519457984325

2.137577296141454
1.5891661015992176
0.39961412719908385
0.40016177215670623
0.36361731249245594
0.3920983724478179
0.3998553984885672
0.3998424782022648
0.38044317313907816
0.39988833692389897
1.024729523659098
0.4003202513656133
0.5498274197038102
0.3999973803514329
0.37981404791272055
0.4002950659802421
1.0500060477853388
0.4002777886160471
0.39988590547541636
0.3982200855649577
0.39956657695612074
0.3418619311896234
0.4003839637777649
0.3996805904380254
0.3906895259580464
0.3962234223025402
0.3865476556163456
0.4000787211776059
0.9252481231497294
0.9770056225979554
0.39986271065582385
0.3437766531260201
0.6276365198780482
0.3999913826030414
0.3826694885383925
0.4000605065022956
0.3978698703734981
0.3998861030420806
0.3999989432568117
0.3998778924730061
0.28693065174715937
0.3784162355473283
0.3537523523427165
0.3832383666930826
0.39961957407396353
0.38725700963989806
0.4003529119699275
0.32795877299473497
0.37646956106895035
0.40037145416694886
0.32405185212976517
0.359390136433368

In [ ]:
input_X = T.tensor3("X")
target_y = T.vector("target Y integer",dtype='float32')
input_shape = [None,len(X_train),len(X_train[0])]
input_layer = L.InputLayer(shape = input_shape,input_var=input_X)

dense = L.DenseLayer(input_layer,num_units=len(X_train),nonlinearity = lasagne.nonlinearities.elu)
#dense = L.batch_norm(dense_1)
dense = L.dropout(dense, p = 0.1)
dense = L.DenseLayer(dense,num_units = round(len(X_train)/2),nonlinearity = lasagne.nonlinearities.elu)
dense_output = L.DenseLayer(dense,num_units = round(len(X_train)/4),nonlinearity = lasagne.nonlinearities.elu)


y_predicted = lasagne.layers.get_output(dense_output)

all_weights = lasagne.layers.get_all_params(dense_output,trainable=True)

#Mean categorical crossentropy as a loss function - similar to logistic loss but for multiclass targets
loss = np.mean(( y_predicted - target_y) ** 2)

#This function computes gradient AND composes weight updates just like you did earlier
updates_sgd = lasagne.updates.sgd(loss, all_weights,learning_rate=0.01)

#function that computes loss and updates weights
train_fun = theano.function([input_X,target_y],loss,updates = updates_sgd)



In [ ]:
import time

num_epochs = 100 #amount of passes through the data
            
batch_size = 50 #number of samples processed at each function call

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train,batch_size):
        inputs, targets = batch
        train_err_batch = train_fun(inputs, targets)
        train_err += train_err_batch
        train_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))


In [ ]:
test_err = 0
test_batches = 0

for batch in iterate_minibatches(X_train, y_train,batch_size):
    inputs, targets = batch
    test_err_batch = train_fun(inputs, targets)
    test_err += test_err_batch
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_err / test_batches * 100))